In [46]:
import openml
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder #For encoding categorical variables
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans,AgglomerativeClustering,DBSCAN
from sklearn.metrics import accuracy_score,f1_score,adjusted_rand_score,silhouette_score
from joblib import Parallel,delayed
import time

# Download Datasets Using openml

In [47]:
iris = openml.datasets.get_dataset("iris")
iris_df, iris_label, categorical_indicator, attribute_names = iris.get_data(
    target=iris.default_target_attribute, dataset_format="dataframe"
)
iris_df["class"]=iris_label
iris_x=iris_df.iloc[:,:4]
iris_df

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [48]:
wine = openml.datasets.get_dataset("wine")
wine_df, wine_label, categorical_indicator, attribute_names = wine.get_data(
    target= wine.default_target_attribute, dataset_format="dataframe"
)
wine_df["class"]=wine_label
wine_x=wine_df.iloc[:,:13]
wine_df

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


# Identify Data Types

In [49]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   sepallength  150 non-null    float64 
 1   sepalwidth   150 non-null    float64 
 2   petallength  150 non-null    float64 
 3   petalwidth   150 non-null    float64 
 4   class        150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [50]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Alcohol                         178 non-null    float64 
 1   Malic_acid                      178 non-null    float64 
 2   Ash                             178 non-null    float64 
 3   Alcalinity_of_ash               178 non-null    float64 
 4   Magnesium                       178 non-null    uint8   
 5   Total_phenols                   178 non-null    float64 
 6   Flavanoids                      178 non-null    float64 
 7   Nonflavanoid_phenols            178 non-null    float64 
 8   Proanthocyanins                 178 non-null    float64 
 9   Color_intensity                 178 non-null    float64 
 10  Hue                             178 non-null    float64 
 11  OD280%2FOD315_of_diluted_wines  178 non-null    float64 
 12  Proline               

# Transform categorical variable to numeric

In [51]:
le=LabelEncoder()

iris_y=le.fit_transform(iris_label)
wine_y=le.fit_transform(wine_label)

# Min-max normalise

In [52]:
scaler = MinMaxScaler()

iris_x_scaled=scaler.fit_transform(iris_x)
wine_x_scaled=scaler.fit_transform(wine_x)


# DataSets

In [53]:
Dataset = ["Iris","Wine"]

# KMeans

In [54]:
def kmeans (x, y, parameters):
    start_time = time.time()
    kmeans = KMeans(n_clusters = parameters[0], max_iter = parameters[1], n_init = parameters[2])
    y_kmeans = kmeans.fit_predict(x)
    kmeans_f1score = f1_score(y, y_kmeans, average = 'weighted')
    kmeans_ars = adjusted_rand_score(y, y_kmeans)
    kmeans_sscore = silhouette_score(x, y_kmeans, metric='euclidean')
    kmeans_execution_time = time.time() - start_time
    return y_kmeans, kmeans_f1score, kmeans_ars, kmeans_sscore, kmeans_execution_time

In [55]:
n_clusters=[2,3,4,5,6,7]
max_iter=[200,300,400]
n_init=[5,10,15]
    
kmean_parameters = pd.DataFrame({"n_clusters":[] , 
                                 "max_iter":[] , 
                                 "n_init":[]}).astype(int)


In [59]:
for i in n_clusters:
        for  ite in max_iter:
            for n in n_init:
                kmean_parameters = kmean_parameters.append({"n_clusters" : i, 
                                                            "max_iter" : ite, 
                                                            "n_init" : n},ignore_index=True)

In [60]:
kmean_parameters

,n_clusters,max_iter,n_init
0,2,200,5
1,2,200,10
2,2,200,15
3,2,300,5
4,2,300,10
...,...,...,...
103,7,300,10
104,7,300,15
105,7,400,5
106,7,400,10


In [61]:
final_iris_kmeans = Parallel(n_jobs=-1)(delayed(kmeans)(iris_x_scaled, iris_y, kmean_parameters.iloc[i]) for i in range(0, len(kmean_parameters)))
final_wine_kmeans = Parallel(n_jobs=-1)(delayed(kmeans)(wine_x_scaled, wine_y, kmean_parameters.iloc[i]) for i in range(0, len(kmean_parameters)))

In [62]:
final_kmeans_df=pd.DataFrame({
    "Dataset":[],
    "n_clusters":[],
    "max_iter":[],
    "n_init":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [63]:
for i in range(0, len(kmean_parameters)):
    final_kmeans_df =final_kmeans_df.append({
        "Dataset":Dataset[0],
        "n_clusters":kmean_parameters.iloc[i][0],
        "max_iter":kmean_parameters.iloc[i][1],
        "n_init":kmean_parameters.iloc[i][2],
        "f1 score":final_iris_kmeans[i][1],
        "Adjusted Random Score":final_iris_kmeans[i][2],
        "Silhouette Score":final_iris_kmeans[i][3],
        "Execution Time":final_iris_kmeans[i][4],
        },ignore_index=True)
    
for i in range(0, len(kmean_parameters)):
    final_kmeans_df =final_kmeans_df.append({
        "Dataset":Dataset[0],
        "n_clusters":kmean_parameters.iloc[i][0],
        "max_iter":kmean_parameters.iloc[i][1],
        "n_init":kmean_parameters.iloc[i][2],
        "f1 score":final_wine_kmeans[i][1],
        "Adjusted Random Score":final_wine_kmeans[i][2],
        "Silhouette Score":final_wine_kmeans[i][3],
        "Execution Time":final_wine_kmeans[i][4],
        "Dataset":Dataset[1]
        },ignore_index=True)
final_kmeans_df

,Dataset,n_clusters,max_iter,n_init,f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,2.0,200.0,5.0,0.000000,0.568116,0.629468,0.017953
1,Iris,2.0,200.0,10.0,0.000000,0.568116,0.629468,0.027926
2,Iris,2.0,200.0,15.0,0.000000,0.568116,0.629468,0.060836
3,Iris,2.0,300.0,5.0,0.000000,0.568116,0.629468,0.017953
4,Iris,2.0,300.0,10.0,0.000000,0.568116,0.629468,0.035908
...,...,...,...,...,...,...,...,...
211,Wine,7.0,300.0,10.0,0.016795,0.457027,0.129570,0.062484
212,Wine,7.0,300.0,15.0,0.036678,0.469308,0.159785,0.093729
213,Wine,7.0,400.0,5.0,0.512297,0.468801,0.142224,0.031241
214,Wine,7.0,400.0,10.0,0.009470,0.467432,0.129503,0.046863


# Agglomerative Clustering

In [64]:
def agglomerative (x, y, parameters):
    start_time = time.time()
    agglomerative= AgglomerativeClustering(n_clusters = parameters[0], linkage = parameters[1])
    y_agglomerative = agglomerative.fit_predict(x)
    agglomerative_f1score = f1_score(y, y_agglomerative, average = "weighted")
    agglomerative_ars = adjusted_rand_score(y, y_agglomerative)
    agglomerative_sscore = silhouette_score(x, y_agglomerative, metric="euclidean")
    agglomerative_execution_time = time.time() - start_time
    return y_agglomerative, agglomerative_f1score,agglomerative_ars, agglomerative_sscore, agglomerative_execution_time

In [77]:
n_clusters =[2,3,4,5,6,7]
linkage = ["ward", "complete", "average", "single"]

agglomerative_parameters = pd.DataFrame({
    "n_clusters":[] ,
    "linkage":[]}).astype(int)
  
for i in n_clusters:
        for  n in linkage:
                agglomerative_parameters = agglomerative_parameters.append({
                    "n_clusters" : i, 
                    "linkage" : n },
                    ignore_index=True)

In [78]:
agglomerative_parameters

,n_clusters,linkage
0,2,ward
1,2,complete
2,2,average
3,2,single
4,3,ward
5,3,complete
6,3,average
7,3,single
8,4,ward
9,4,complete


In [79]:
final_iris_aggromilative = Parallel(n_jobs=-1)(delayed(agglomerative)(iris_x_scaled, iris_y, agglomerative_parameters.iloc[i]) for i in range(0, len(agglomerative_parameters)))
final_wine_aggromilative = Parallel(n_jobs=-1)(delayed(agglomerative)(wine_x_scaled, wine_y, agglomerative_parameters.iloc[i]) for i in range(0, len(agglomerative_parameters)))

In [80]:
final_aggromilative_df=pd.DataFrame({
    "Dataset":[],
    "n_clusters":[],
    "linkage":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [81]:
for i in range(0, len(agglomerative_parameters)):
    final_aggromilative_df =final_aggromilative_df.append({
        "Dataset":Dataset[0],
        "n_clusters":agglomerative_parameters.iloc[i][0],
        "linkage":agglomerative_parameters.iloc[i][1],
        "f1 score":final_iris_aggromilative[i][1],
        "Adjusted Random Score":final_iris_aggromilative[i][2],
        "Silhouette Score":final_iris_aggromilative[i][3],
        "Execution Time":final_iris_aggromilative[i][4],
        },ignore_index=True)
    
for i in range(0, len(agglomerative_parameters)):
    final_aggromilative_df =final_aggromilative_df.append({
        "Dataset":Dataset[1],
        "n_clusters":agglomerative_parameters.iloc[i][0],
        "linkage":agglomerative_parameters.iloc[i][1],
        "f1 score":final_wine_aggromilative[i][1],
        "Adjusted Random Score":final_wine_aggromilative[i][2],
        "Silhouette Score":final_wine_aggromilative[i][3],
        "Execution Time":final_wine_aggromilative[i][4],
        },ignore_index=True)
    
final_aggromilative_df

,Dataset,n_clusters,linkage,f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,2.0,ward,0.000000,0.568116,0.629468,0.031237
1,Iris,2.0,complete,0.208740,0.223420,0.303913,0.031250
2,Iris,2.0,average,0.000000,0.568116,0.629468,0.015630
3,Iris,2.0,single,0.555556,0.568116,0.629468,0.015623
4,Iris,3.0,ward,0.265060,0.719584,0.504349,0.015622
5,Iris,3.0,complete,0.438971,0.706006,0.503067,0.000000
6,Iris,3.0,average,0.883294,0.719584,0.504349,0.015618
7,Iris,3.0,single,0.000000,0.558371,0.530889,0.000000
8,Iris,4.0,ward,0.825060,0.645423,0.432557,0.015622
9,Iris,4.0,complete,0.007937,0.578129,0.393187,0.000000


# DBScan Clustering

In [183]:
def dbscan (x, y, parameters):
    start_time = time.time()
    dbscan= DBSCAN(eps = parameters[0], min_samples = parameters[1])
    y_dbscan = dbscan.fit_predict(x)
    dbscan_f1score = f1_score(y, y_dbscan, average = "weighted")
    dbscan_ars = adjusted_rand_score(y, y_dbscan)
    dbscan_n_clusters = len(set(y_dbscan)) - (1 if -1 in y_dbscan else 0)
    if dbscan_n_clusters>=2:
        dbscan_sscore = silhouette_score(x, y_dbscan, metric="euclidean")
    else:
        dbscan_sscore = print("111")
    dbscan_execution_time = time.time() - start_time
    return y_dbscan, dbscan_f1score,dbscan_ars,dbscan_sscore, dbscan_execution_time

In [184]:
eps = [0.41,0.42,0.43,0.44,0.45,0.46]
min_samples = [2,3,4,5,6,7,8]

dbscan_parameters = pd.DataFrame({
    "eps":[],
    "min_samples" :[]
}).astype(int)

for i in eps:
    for n in min_samples:
        dbscan_parameters=dbscan_parameters.append({
            "eps" : i,
            "min_samples": n
        },ignore_index=True)
        
dbscan_parameters

,eps,min_samples
0,0.41,2.0
1,0.41,3.0
2,0.41,4.0
3,0.41,5.0
4,0.41,6.0
5,0.41,7.0
6,0.41,8.0
7,0.42,2.0
8,0.42,3.0
9,0.42,4.0


In [185]:
final_iris_dbscan = Parallel(n_jobs=-1)(delayed(dbscan)(iris_x_scaled, iris_y, dbscan_parameters.iloc[i]) for i in range(0, len(dbscan_parameters)))

In [186]:
final_dbscan_df=pd.DataFrame({
    "Dataset":[],
    "eps":[],
    "min_samples":[],
    "f1 score":[],
    "Adjusted Random Score":[],
    "Silhouette Score":[],
    "Execution Time":[],
})

In [187]:
for i in range(0, len(dbscan_parameters)):
    final_dbscan_df =final_dbscan_df.append({
        "Dataset":Dataset[0],
        "eps":dbscan_parameters.iloc[i][0],
        "min_samples":dbscan_parameters.iloc[i][1],
        "f1 score":final_iris_dbscan[i][1],
        "Adjusted Random Score":final_iris_dbscan[i][2],
        "Silhouette Score":final_iris_dbscan[i][3],
        "Execution Time":final_iris_dbscan[i][4],
        },ignore_index=True)

In [188]:
final_dbscan_df

,Dataset,eps,min_samples,f1 score,Adjusted Random Score,Silhouette Score,Execution Time
0,Iris,0.41,2.0,0.555556,0.568116,0.629468,0.006982
1,Iris,0.41,3.0,0.555556,0.568116,0.629468,0.006980
2,Iris,0.41,4.0,0.555556,0.568116,0.629468,0.007980
3,Iris,0.41,5.0,0.555556,0.568116,0.629468,0.007981
4,Iris,0.41,6.0,0.555556,0.568116,0.629468,0.007978
5,Iris,0.41,7.0,0.555556,0.568116,0.629468,0.007977
6,Iris,0.41,8.0,0.555556,0.568116,0.629468,0.008976
7,Iris,0.42,2.0,0.555556,0.568116,0.629468,0.018948
8,Iris,0.42,3.0,0.555556,0.568116,0.629468,0.006982
9,Iris,0.42,4.0,0.555556,0.568116,0.629468,0.006979


In [182]:
# n_clusters = len(set(y_dbscan)) - (1 if -1 in y_dbscan else 0)
# n_noise = list(y_dbscan).count(-1)

# print('Estimated number of clusters: %d' % n_clusters)
# print('Estimated number of noise points: %d' % n_noise)

In [ ]:
colors = ['mediumorchid', 'maroon', 'forestgreen','navy','goldenrod']
vectorizer = np.vectorize(lambda x: colors[x % len(colors)])

plt.scatter(x_scaled[:,0], x_scaled[:,1], c=vectorizer(y_dbscan))

Unlike k-means, DBSCAN will figure out the number of clusters. 
DBSCAN works by determining whether the minimum number of points are close enough to 
one another to be considered part of a single cluster. DBSCAN is very sensitive to scale since 
epsilon is a fixed value for the maximum distance between two points.

# Optics Clustering

# Gaussian mixtures Clustering

# Affinity propagation

In [ ]:
# from sklearn.cluster import AffinityPropagation

# afp = AffinityPropagation(damping=0.9, max_iter=200, convergence_iter=15, copy=True, preference=-5, affinity='euclidean', verbose=False, random_state=None)
# y_afp=afp.fit_predict(x_scaled)
# y_afp

# Mean-shift 

# Spectral Clustering

# Ward hierarchical

In [ ]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 0, 1, 1, 2]
metrics.rand_score(labels_true, labels_pred)